In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing required packages
import pandas as pd
import numpy as np

import os
import gc


# import ipywidgets as widgets
import warnings
warnings.filterwarnings("ignore")

In [4]:
# from ta_lib.core.api import load_yml, create_context,_change_permissions_recursive

In [5]:
# Defining configs
config_path = '../conf/config.yml'
config = create_context(config_path)
cfg = config.offline

In [25]:
item_master = pd.read_csv(cfg['ITEM_MASTER'].format(**cfg))

In [26]:
df_dig_promo_fun = pd.read_excel(cfg['DIGITAL_OFFER_FUNNEL'].format(**cfg))

In [27]:
item_group = dict(zip(item_master.Group_id, item_master['Product Group']))

In [28]:
# item_group

In [29]:
df_dig_promo_fun.columns

Index(['offer_id', 'sld_menu_itm_id', 'product_price', 'products_quantity',
       'discount_dimension', 'Product Groupings', 'name from excel cal',
       'min_date', 'max_date', 'Group_id', 'Name', 'child_item_flag',
       'Product Group', 'day_part'],
      dtype='object')

In [30]:
offer_grp_map = df_dig_promo_fun.groupby(['offer_id','Product Groupings']).size().reset_index()

In [31]:
seg_disc_path = '/opt/sasdata/dev/PricingEngines/Engine/DiscountEngines/Russia/data/processed/digital/segmentation_discount_qty/company=mcopco/'
seg_qty_path = '/opt/sasdata/dev/PricingEngines/Engine/DiscountEngines/Russia/data/processed/digital/segmentation_quantity_company/grouped/'

In [32]:
df_seg_disc = pd.read_parquet(seg_disc_path)

# df_promo_disc = df_seg_disc.groupby(["sld_menu_itm_id","pos_busn_dt","offer_id"]).pos_itm_dsc_qt.sum().reset_index()

df_seg_disc = df_seg_disc.merge(offer_grp_map[['offer_id','Product Groupings']], on= ['offer_id'], how = 'left')

df_seg_disc['Product Group'] = df_seg_disc.sld_menu_itm_id.map(item_group)

df_seg_disc.drop(['pos_itm_dsc_amt','cutoff'], axis = 1, inplace = True)

In [33]:
seg_qty = '../../notebooks/offline/akash_dev/seg_item_qty.parquet'

In [34]:
df_seg_qty_1 = pd.read_parquet(seg_qty)

In [35]:
df_seg_qty_1 = df_seg_qty_1[df_seg_qty_1.company == 'mcopco']

In [36]:
df_seg_qty_1

,pos_busn_dt,segments,quantity,monthid,sld_menu_itm_id,company,cutoff,lag,duration
159,2020-12-07,Lapsing Users - At least one transaction,1,202012,10000106,mcopco,20201130,7,7
160,2020-12-10,Active Users - High frequency High offers,1,202012,10000106,mcopco,20201130,7,7
161,2020-12-11,Active Users - High frequency High offers,1,202012,10000106,mcopco,20201130,7,7
162,2020-12-12,Active Users - High frequency Low offers High ...,1,202012,10000106,mcopco,20201130,7,7
163,2020-12-14,Active Users - High frequency High offers,1,202012,10000106,mcopco,20201207,7,7
...,...,...,...,...,...,...,...,...,...
3707322,2021-05-10,Active Users - High frequency Low offers High ...,2,202105,9569,mcopco,20210503,7,7
3707323,2021-05-10,Active Users - High frequency Low offers Low c...,1,202105,9569,mcopco,20210503,7,7
3707324,2021-05-10,Lapsing Users - At least one transaction,1,202105,9569,mcopco,20210503,7,7
3707325,2021-05-10,Pre-registration,1,202105,9569,mcopco,20210503,7,7


In [37]:
df_seg_disc_1 = df_seg_disc.merge(df_seg_qty_1[['pos_busn_dt','segments','sld_menu_itm_id','quantity']], how = 'left', on = ['pos_busn_dt','segments','sld_menu_itm_id'])

In [38]:
df_seg_disc_1

,sld_menu_itm_id,pos_busn_dt,offer_id,segments,pos_itm_dsc_qt,Product Groupings,Product Group,quantity
0,1022,2020-12-15,1089.0,Active Users - High frequency High offers,23,Set - 2 products,Breakfast,95.0
1,1022,2020-12-15,1089.0,Active Users - High frequency Low offers High ...,1,Set - 2 products,Breakfast,48.0
2,1022,2020-12-15,1089.0,Active Users - High frequency Low offers Low c...,2,Set - 2 products,Breakfast,34.0
3,1022,2020-12-15,1089.0,Active Users - Low frequency,11,Set - 2 products,Breakfast,61.0
4,1022,2020-12-15,1089.0,Enrollment - Late start,1,Set - 2 products,Breakfast,3.0
...,...,...,...,...,...,...,...,...
60226,7163,2021-11-28,2384.0,Inactive Users - At least one transaction,36,Set - 2 products,Cold Drinks,145.0
60227,7163,2021-11-28,2384.0,Inactive Users - No transactions,10,Set - 2 products,Cold Drinks,43.0
60228,7163,2021-11-28,2384.0,Lapsing Users - At least one transaction,47,Set - 2 products,Cold Drinks,143.0
60229,7163,2021-11-28,2384.0,Not downloaded,15,Set - 2 products,Cold Drinks,49.0


In [39]:
df_seg_disc_1.to_excel("segment_discount_qty.xlsx", index = False)

In [ ]:
df_seg_qty = pd.read_parquet(seg_qty_path)

In [ ]:
df_seg_qty.to_parquet("segment_level_qty.parquet")

In [ ]:
df_seg_qty[df_seg_qty.company == 'mcopco']